## Setting enviroment

Obtaining language model from huggingface


In [ ]:
!huggingface-cli download bartowski/Llama-3.2-3B-Instruct-GGUF  --include "Llama-3.2-3B-Instruct-f16.gguf" --local-dir ./

Installing langchain and langchain comunity

In [ ]:
!pip install langchain
!pip install langchain-community

Installing gradio

Imports

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp


In [1]:
import gradio as gr
import whisper
import os
import torch

## Implementation

Configuring the language model

load whisper and deepseek

In [4]:
transcription_model = whisper.load_model("base")

d:\TESIS\System\.venv\Lib\site-packages\whisper\__init__.py:69: UserWarning: C:\Users\lin\.cache\whisper\base.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|████████████████████████████████████████| 139M/139M [05:02<00:00, 480kiB/s]


In [ ]:
llm = LlamaCpp(
    model_path="/content/Llama-3.2-3B-Instruct-f16.gguf",
    temperature=0.8,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

In [33]:

import gradio as gr

def transcribe_audio(audio_file):
    # result = transcription_model.transcribe(audio_file)
    # transcription = result["text"]
    # return transcription
    return 'This is a transcription example...'

def edit_transcription(text):
    return text

def delete_transcription():
    return "", False


def audio_interface(audio):
    assert audio, 'Audio should be defined'
 
    if audio is None:
        return "Please upload an audio file.", False
#add error handlers with the transcription
    transcription = transcribe_audio(audio)

    return transcription, transcription, gr.update(interactive=True), gr.update(interactive=True) 


def edit_mode(edit_mode):
    return gr.update(visible=True),gr.update(visible=True),gr.update(visible=False),gr.update(visible=False), gr.update(visible=False)


def save_changes(edited_text, old_text):
    if old_text != edited_text:
        return gr.update(edited_text, visible=False), gr.update(edited_text, visible=True), gr.update(visible=False), gr.update(visible=True),gr.update(visible=True)
    else: 
        return gr.update(old_text, visible=False), gr.update(old_text, visible=True), gr.update(visible=False), gr.update(visible=True),gr.update(visible=True) 


def cancel_edit(old_text):
    return gr.update(old_text, visible=False), gr.update(old_text, visible=True), gr.update(visible=False), gr.update(visible=True),gr.update(visible=True)




with gr.Blocks() as demo:
    gr.Markdown("### Audio Transcription Interface")


    audio_input = gr.Audio(label="Upload Audio", type="filepath", interactive=True) #done
        
    transcribe_button = gr.Button("Transcribe Audio", interactive=False) #done
    
    transcription_text_show = gr.Markdown(label="Transcription")

    transcription_text_editable = gr.Textbox(label="Transcription", visible=False)

    with gr.Row():
        edit_button = gr.Button("Edit Transcription", interactive=False) #done
        delete_button = gr.Button("Delete Transcription", interactive=False) #pending

    with gr.Row(visible=False) as edit_controls:
        save_button = gr.Button("Save Changes") 
        cancel_edit_button = gr.Button("Cancel Edit")

    # Update transcription button state based on audio upload
    audio_input.change(
        fn=lambda audio: gr.update(interactive=audio is not None),
        inputs=audio_input,
        outputs=transcribe_button
    )
    # Perform transcription
    transcribe_button.click(
        fn=audio_interface,
        inputs=audio_input,
        outputs=[transcription_text_editable, transcription_text_show, edit_button, delete_button]
    )

    # Enable edit mode
    edit_button.click(
        fn=lambda: edit_mode(True),
        outputs=[edit_controls, transcription_text_editable, edit_button, delete_button, transcription_text_show]
    )
    
    # Save changes
    save_button.click(
        fn=save_changes,
        inputs=[transcription_text_editable, transcription_text_show],
        outputs=[transcription_text_editable, transcription_text_show, edit_controls, edit_button, delete_button]
    )

    # Cancel edit
    cancel_edit_button.click(
        fn=cancel_edit,
        inputs=transcription_text_show,
        outputs=[transcription_text_editable, transcription_text_show, edit_controls, edit_button, delete_button]
    )

    # Delete transcription
    delete_button.click(
        fn=delete_transcription,
        outputs=[transcription_text_editable, edit_button]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [6]:
import gradio as gr

def transcribe_audio(audio_file):
    # result = transcription_model.transcribe(audio_file)
    # transcription = result["text"]
    # return transcription
    return 'This is a transcription example...'

def edit_transcription(text):
    return text


def audio_interface(audio):
    if audio is None:
        return "Please upload an audio file.", False

    transcription = transcribe_audio(audio)

    return transcription, True


def update_transcription(transcription, edit_mode):
    if edit_mode:
        return gr.update(interactive=True)
    else:
        return gr.update(interactive=False)


def save_changes(transcription, edited_text):
    return edited_text, False


def cancel_edit(transcription):
    return transcription, False


def delete_transcription():
    return "", False

with gr.Blocks() as demo:
    gr.Markdown("### Audio Transcription Interface")

    with gr.Row():
        audio_input = gr.Audio(label="Upload Audio", type="filepath", interactive=True)
        cancel_button = gr.Button("Cancel Upload", interactive=False)

    transcribe_button = gr.Button("Transcribe Audio", interactive=False)
    transcription_text_editable = gr.Textbox(label="Transcription", interactive=False)

    with gr.Row():
        edit_button = gr.Button("Edit Transcription", interactive=False)
        delete_button = gr.Button("Delete Transcription", interactive=False)

    with gr.Row(visible=False) as edit_controls:
        save_button = gr.Button("Save Changes")
        cancel_edit_button = gr.Button("Cancel Edit")

    # Update transcription button state based on audio upload
    audio_input.change(
        fn=lambda audio: gr.update(interactive=audio is not None),
        inputs=audio_input,
        outputs=transcribe_button
    )

    # Perform transcription
    transcribe_button.click(
        fn=audio_interface,
        inputs=audio_input,
        outputs=[transcription_text_editable, edit_button]
    )

    # Enable edit mode
    edit_button.click(
        fn=lambda: gr.update(visible=True),
        outputs=edit_controls
    ).then(
        fn=lambda: gr.update(interactive=True),
        outputs=transcription_text_editable
    )

    # Save changes
    save_button.click(
        fn=save_changes,
        inputs=[transcription_text_editable, transcription_text_editable],
        outputs=[transcription_text_editable, edit_controls]
    )

    # Cancel edit
    cancel_edit_button.click(
        fn=cancel_edit,
        inputs=transcription_text_editable,
        outputs=[transcription_text_editable, edit_controls]
    )

    # Delete transcription
    delete_button.click(
        fn=delete_transcription,
        outputs=[transcription_text_editable, edit_button]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
